In [2]:
import numpy as np
import pandas as pd
import math
import requests
import bs4
from bs4 import BeautifulSoup

In [11]:
## Import csv of company listing we extracted in previous scripts
df = pd.read_csv('./datasets/companylisting.csv', usecols=['ticker']).values

In [3]:
## Function for retrieving fundamental company financials using web scraping looking at revenues, expenses, and profitabilities
def get_financials(tickers):
    financials = []
    if type(tickers) == list or type(tickers) == np.ndarray:
        for t in tickers:
            year = np.nan
            last_year = np.nan
            notice = np.nan
            rev = np.nan
            rev_ly = np.nan
            cogs = np.nan
            cogs_ly = np.nan
            gross_profit = np.nan
            gross_profit_ly = np.nan
            operating_expense = np.nan
            operating_expense_ly = np.nan
            operating_profit = np.nan
            operating_profit_ly = np.nan
            net_profit = np.nan
            net_profit_ly = np.nan
            try:
                r = requests.get('https://ca.finance.yahoo.com/quote/{}/financials?p={}'.format(t.replace('.', '-'),t.replace('.', '-')))
                soup = bs4.BeautifulSoup(r.text, "html.parser")

                year = soup.findAll('td', {'class':'C($gray) Ta(end)'})[0].text
                last_year = soup.findAll('td', {'class':'C($gray) Ta(end)'})[1].text
                notice = soup.findAll('span', {'Fz(xs) C($gray) Mstart(25px) smartphone_Mstart(0px) smartphone_D(b) smartphone_Mt(5px)'})[0].text
                rev = soup.findAll('td', {'class':'Fz(s) Ta(end) Pstart(10px)'})[0].text
                rev_ly = soup.findAll('td', {'class':'Fz(s) Ta(end) Pstart(10px)'})[1].text
                cogs = soup.findAll('td', {'class':'Fz(s) Ta(end) Pstart(10px)'})[4].text
                cogs_ly = soup.findAll('td', {'class':'Fz(s) Ta(end) Pstart(10px)'})[5].text
                gross_profit = soup.findAll('td', {'class':'Fw(600) Fz(s) Ta(end) Pb(20px)'})[0].text
                gross_profit_ly = soup.findAll('td', {'class':'Fw(600) Fz(s) Ta(end) Pb(20px)'})[1].text
                operating_expense = soup.findAll('td', {'class':"Fz(s) Ta(end) Pstart(10px)"})[24].text
                operating_expense_ly = soup.findAll('td', {'class':"Fz(s) Ta(end) Pstart(10px)"})[25].text
                operating_profit = soup.findAll('td', {'class':'Fw(600) Fz(s) Ta(end) Pb(20px)'})[4].text
                operating_profit_ly = soup.findAll('td', {'class':'Fw(600) Fz(s) Ta(end) Pb(20px)'})[5].text
                net_profit = soup.findAll('td', {'class':'Fw(600) Ta(end) Py(8px) Pt(36px)'})[0].text
                net_profit_ly = soup.findAll('td', {'class':'Fw(600) Ta(end) Py(8px) Pt(36px)'})[1].text

                financials.append([t, year, last_year, notice, rev, rev_ly, cogs, cogs_ly, gross_profit, gross_profit_ly, operating_expense, operating_expense_ly, operating_profit, operating_profit_ly, net_profit, net_profit_ly])
            except:
                print('Error has Occured. Could not Retrieve Data for {}'.format(t))
                financials.append([t, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
    else:
        return 'Error: Input Type Not-a-list'
    
    return financials

In [4]:
# Function call for retrieving fundamental data of company balance sheets, mostly to do with assets, liabilities, and equities
def get_balance_sheets(tickers):
    balance_sheets = []
    if type(tickers) == list or type(tickers) == np.ndarray:
        for t in tickers:
            date = np.nan
            notice = np.nan
            shareholders_equity = np.nan
            total_equity = np.nan
            intangible_assets = np.nan
            total_assets = np.nan
            total_liabilities = np.nan
            try:
                r = requests.get('https://www.marketwatch.com/investing/stock/{}/financials/balance-sheet/quarter'.format(t.replace('-', '.')))
                soup = bs4.BeautifulSoup(r.text, "html.parser")
                notice = soup.findAll('th')[0].text.strip()
                date = soup.findAll('th')[5].text.strip()

                for i in range(0,len(soup.findAll('tr', {'class':'partialSum'}))):
                    if soup.findAll('tr', {'class':'partialSum'})[i].findChildren()[0].text.strip() == "Total Shareholders' Equity":
                        shareholders_equity = soup.findAll('tr', {'class':'partialSum'})[i].findChildren()[7].text.strip()
                    elif soup.findAll('tr', {'class':'partialSum'})[i].findChildren()[0].text.strip() == 'Total Equity':
                        total_equity = soup.findAll('tr', {'class':'partialSum'})[i].findChildren()[5].text.strip()
                    else:
                        continue
                for i in range(0,len(soup.findAll('tr', {'class':'rowLevel-2'}))):
                    if soup.findAll('tr', {'class':'rowLevel-2'})[i].findChildren()[0].text.strip() == 'Intangible Assets':
                        intangible_assets = soup.findAll('tr', {'class':'rowLevel-2'})[i].findChildren()[5].text.strip()
                    else:
                        continue
                for i in range(0,len(soup.findAll('tr', {'class':'mainRow'}))):
                    if soup.findAll('tr', {'class':'mainRow'})[i].findChildren()[0].text.strip() == 'Intangible Assets':
                        intangible_assets = soup.findAll('tr', {'class':'mainRow'})[i].findChildren()[5].text.strip()
                    else:
                        continue
                for i in range(0,len(soup.findAll('tr', {'class':'totalRow'}))):
                    if soup.findAll('tr', {'class':'totalRow'})[i].findChildren()[0].text.strip() == 'Total Assets':
                        total_assets = soup.findAll('tr', {'class':'totalRow'})[i].findChildren()[7].text.strip()
                    elif soup.findAll('tr', {'class':'totalRow'})[i].findChildren()[0].text.strip() == 'Total Liabilities':
                        total_liabilities = soup.findAll('tr', {'class':'totalRow'})[i].findChildren()[7].text.strip()
                    else:
                        continue
                balance_sheets.append([t, date, notice, total_assets, total_liabilities, intangible_assets, shareholders_equity, total_equity])
            except:
                print('Error has Occured. Could not Retrieve Data for {}'.format(t))
                balance_sheets.append([t, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
    else:
        return 'Error: Input Type Not-a-list'
    
    return balance_sheets

In [5]:
## Company's most recent quarter end results in the income statement
def get_quarter_ends(tickers):
    quarter_ends = []
    if type(tickers) == list or type(tickers) == np.ndarray:
        date = np.nan
        notice = np.nan
        revenue = np.nan
        gross_profit = np.nan
        operating_profit = np.nan
        net_income = np.nan
        
        for t in tickers:
            try:
                r = requests.get('https://www.barchart.com/stocks/quotes/{}/income-statement/quarterly'.format(t.replace('-', '.')))
                soup = bs4.BeautifulSoup(r.text, "html.parser")

                if soup.findAll('td', {'class':'bc-financial-report__cell-group-field-label'})[0].text.strip() == 'Interest Income':
                    for i in range(0,len(soup.findAll('tr'))):
                        if soup.findAll('tr')[i].findChildren()[0].text.strip().lower() == 'sales':
                            revenue = soup.findAll('tr')[i].findChildren()[1].text.strip()
                        elif soup.findAll('tr')[i].findChildren()[0].text.strip().lower() == 'interest expense (net of interest income)':
                            gross_profit = soup.findAll('tr')[i].findChildren()[1].text.strip()
                        elif soup.findAll('tr')[i].findChildren()[0].text.strip().lower() == 'pre-tax income':
                            operating_profit = soup.findAll('tr')[i].findChildren()[1].text.strip()
                elif soup.findAll('td', {'class':'bc-financial-report__cell-group-field-label'})[0].text.strip() == 'Sales':
                    for i in range(0,len(soup.findAll('tr'))):
                        if soup.findAll('tr')[i].findChildren()[0].text.strip().lower() == 'sales':
                            revenue = soup.findAll('tr')[i].findChildren()[1].text.strip()
                        elif soup.findAll('tr')[i].findChildren()[0].text.strip().lower() == 'gross profit':
                            gross_profit = soup.findAll('tr')[i].findChildren()[1].text.strip()
                        elif soup.findAll('tr')[i].findChildren()[0].text.strip().lower() == 'operating income $m':
                            operating_profit = soup.findAll('tr')[i].findChildren()[1].text.strip()
                if type(operating_profit)==float:
                    for i in range(0,len(soup.findAll('tr'))):
                        if soup.findAll('tr')[i].findChildren()[0].text.strip().lower() == 'pre-tax income':
                            operating_profit = soup.findAll('tr')[i].findChildren()[1].text.strip()
                        else:
                            continue
                net_income = soup.findAll('tr', {'class':'bc-financial-report__row-group-total'})[0].findChildren()[1].text.strip()
                notice = soup.findAll('span', {'class':'bc-financial-report__notice'})[0].text
                date = soup.findAll('tr', {'class':'bc-financial-report__row-dates'})[0].findChildren()[1].text.strip()

                quarter_ends.append([t, date, notice, revenue, gross_profit, operating_profit, net_income])

            except:
                print('Error has Occured. Could not Retrieve Data for {}'.format(t))
                quarter_ends.append([t, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
            
    else:
        return 'Error: Input Type Not-a-list'
    
    return quarter_ends

In [6]:
## Getting the analyst ratings of companies
def get_ratings(tickers):
    ratings = []
    if type(tickers) == list or type(tickers) == np.ndarray:
        for t in tickers:
            rating = np.nan
            try:
                r = requests.get('https://www.barchart.com/stocks/quotes/{}/analyst-ratings'.format(t.replace('-', '.')))
                soup = bs4.BeautifulSoup(r.text, "html.parser")
                rating = soup.findAll('div', {'class': 'block__colored-header rating'})[3].text.strip()
                ratings.append([t, rating])
                
            except:
                print('Error has Occured. Could not Retrieve Data for {}'.format(t))
                ratings.append([t, np.nan])
    else:
        return 'Error: Input Type Not-a-list'
    
    return ratings

In [7]:
tickers = df['ticker'].values

In [8]:
## Web Scraping in batches
batchA = tickers[:50]
batchB = tickers[50:100]
batchC = tickers[100:150]
batchD = tickers[150:200]
batchE = tickers[200:250]
batchF = tickers[250:300]
batchG = tickers[300:350]
batchH = tickers[350:400]
batchI = tickers[400:450]
batchJ = tickers[450:500]
batchK = tickers[500:550]
batchL = tickers[550:600]
batchM = tickers[600:650]
batchN = tickers[650:700]
batchO = tickers[700:]

In [9]:
financialsA = get_financials(batchA)
balanceSheetA = get_balance_sheets(batchA)
quarterEndA = get_quarter_ends(batchA)
ratingsA = get_ratings(batchA)

Error has Occured. Could not Retrieve Data for ABMD
Error has Occured. Could not Retrieve Data for AMCR
Error has Occured. Could not Retrieve Data for AATDF


In [10]:
financialsB = get_financials(batchB)
balanceSheetB = get_balance_sheets(batchB)
quarterEndB = get_quarter_ends(batchB)
ratingsB = get_ratings(batchB)

In [11]:
financialsC = get_financials(batchC)
balanceSheetC = get_balance_sheets(batchC)
quarterEndC = get_quarter_ends(batchC)
ratingsC = get_ratings(batchC)

Error has Occured. Could not Retrieve Data for CHTR
Error has Occured. Could not Retrieve Data for BRRAY
Error has Occured. Could not Retrieve Data for BTLCY
Error has Occured. Could not Retrieve Data for BYPLF
Error has Occured. Could not Retrieve Data for CCLAY
Error has Occured. Could not Retrieve Data for CICOY


In [12]:
financialsD = get_financials(batchD)
balanceSheetD = get_balance_sheets(batchD)
quarterEndD = get_quarter_ends(batchD)
ratingsD = get_ratings(batchD)

Error has Occured. Could not Retrieve Data for CTVA


In [13]:
financialsE = get_financials(batchE)
balanceSheetE = get_balance_sheets(batchE)
quarterEndE = get_quarter_ends(batchE)
ratingsE = get_ratings(batchE)

Error has Occured. Could not Retrieve Data for DRE
Error has Occured. Could not Retrieve Data for DXC
Error has Occured. Could not Retrieve Data for EVRG
Error has Occured. Could not Retrieve Data for ENQUF


In [14]:
financialsF = get_financials(batchF)
balanceSheetF = get_balance_sheets(batchF)
quarterEndF = get_quarter_ends(batchF)
ratingsF = get_ratings(batchF)

Error has Occured. Could not Retrieve Data for EVRZF
Error has Occured. Could not Retrieve Data for GFI
Error has Occured. Could not Retrieve Data for GLAPY


In [15]:
financialsG = get_financials(batchG)
balanceSheetG = get_balance_sheets(batchG)
quarterEndG = get_quarter_ends(batchG)
ratingsG = get_ratings(batchG)

Error has Occured. Could not Retrieve Data for HRZN
Error has Occured. Could not Retrieve Data for INFO
Error has Occured. Could not Retrieve Data for HMSNF
Error has Occured. Could not Retrieve Data for HSNGY


In [16]:
financialsH = get_financials(batchH)
balanceSheetH = get_balance_sheets(batchH)
quarterEndH = get_quarter_ends(batchH)
ratingsH = get_ratings(batchH)

Error has Occured. Could not Retrieve Data for IQV
Error has Occured. Could not Retrieve Data for JCI
Error has Occured. Could not Retrieve Data for LHX
Error has Occured. Could not Retrieve Data for JDWPY
Error has Occured. Could not Retrieve Data for KGFHY


In [17]:
financialsI = get_financials(batchI)
balanceSheetI = get_balance_sheets(batchI)
quarterEndI = get_quarter_ends(batchI)
ratingsI = get_ratings(batchI)

Error has Occured. Could not Retrieve Data for LSGOF
Error has Occured. Could not Retrieve Data for MBNKF


In [18]:
financialsJ = get_financials(batchJ)
balanceSheetJ = get_balance_sheets(batchJ)
quarterEndJ = get_quarter_ends(batchJ)
ratingsJ = get_ratings(batchJ)

Error has Occured. Could not Retrieve Data for NILSY
Error has Occured. Could not Retrieve Data for NPCPF


In [19]:
financialsK = get_financials(batchK)
balanceSheetK = get_balance_sheets(batchK)
quarterEndK = get_quarter_ends(batchK)
ratingsK = get_ratings(batchK)

Error has Occured. Could not Retrieve Data for PUBGY
Error has Occured. Could not Retrieve Data for RBGLY


In [20]:
financialsL = get_financials(batchL)
balanceSheetL = get_balance_sheets(batchL)
quarterEndL = get_quarter_ends(batchL)
ratingsL = get_ratings(batchL)

Error has Occured. Could not Retrieve Data for REGN
Error has Occured. Could not Retrieve Data for ROP
Error has Occured. Could not Retrieve Data for SNPS
Error has Occured. Could not Retrieve Data for SBAC
Error has Occured. Could not Retrieve Data for RNSHF
Error has Occured. Could not Retrieve Data for RYCEY
Error has Occured. Could not Retrieve Data for SAGKF
Error has Occured. Could not Retrieve Data for SAPMF
Error has Occured. Could not Retrieve Data for SCBFF
Error has Occured. Could not Retrieve Data for SMGZY
Error has Occured. Could not Retrieve Data for SNPTF


In [21]:
financialsM = get_financials(batchM)
balanceSheetM = get_balance_sheets(batchM)
quarterEndM = get_quarter_ends(batchM)
ratingsM = get_ratings(batchM)

Error has Occured. Could not Retrieve Data for TAP
Error has Occured. Could not Retrieve Data for SOCLF
Error has Occured. Could not Retrieve Data for SPKKY
Error has Occured. Could not Retrieve Data for TMNSF


In [22]:
financialsN = get_financials(batchN)
balanceSheetN = get_balance_sheets(batchN)
quarterEndN = get_quarter_ends(batchN)
ratingsN = get_ratings(batchN)

Error has Occured. Could not Retrieve Data for UDR
Error has Occured. Could not Retrieve Data for VLEEY


In [23]:
financialsO = get_financials(batchO)
balanceSheetO = get_balance_sheets(batchO)
quarterEndO = get_quarter_ends(batchO)
ratingsO = get_ratings(batchO)

Error has Occured. Could not Retrieve Data for WOPEY
Error has Occured. Could not Retrieve Data for WTKWY


In [24]:
## Concatenate all the batches
financials = np.concatenate((financialsA, financialsB, financialsC, financialsD, financialsE, financialsF, financialsG, financialsH, financialsI, financialsJ, financialsK, financialsL, financialsM, financialsN, financialsO), axis=0)
balanceSheet = np.concatenate((balanceSheetA, balanceSheetB, balanceSheetC, balanceSheetD, balanceSheetE, balanceSheetF, balanceSheetG, balanceSheetH, balanceSheetI, balanceSheetJ, balanceSheetK, balanceSheetL, balanceSheetM, balanceSheetN, balanceSheetO), axis=0)
quarterEnd = np.concatenate((quarterEndA, quarterEndB, quarterEndC, quarterEndD, quarterEndE, quarterEndF, quarterEndG, quarterEndH, quarterEndI, quarterEndJ, quarterEndK, quarterEndL, quarterEndM, quarterEndN, quarterEndO), axis=0)
ratings = np.concatenate((ratingsA, ratingsB, ratingsC, ratingsD, ratingsE, ratingsF, ratingsG, ratingsH, ratingsI, ratingsJ, ratingsK, ratingsL, ratingsM, ratingsN, ratingsO), axis=0)

In [25]:
## Export into a csv file format
column_names = ['ticker', 'year', 'last_year', 'notice', 'revenue', 'revenue_ly', 'cogs', 'cogs_ly', 'gross_profit', 'gross_profit_ly', 'operating_expense', 'operating_expense_ly', 'operating_profit', 'operating_profit_ly', 'net_profit', 'net_profit_ly']
financials = pd.DataFrame(data=financials, columns=column_names)
financials.to_csv('./datasets/company_financials.csv')

In [26]:
## Export into a csv file format
column_names = ['ticker', 'quarter_end', 'notice', 'total_assets', 'total_liabilities', 'intangible_assets', 'shareholders_equity', 'total_equity']
balanceSheet = pd.DataFrame(data=balanceSheet, columns=column_names)
balanceSheet.to_csv('./datasets/company_balance_sheets.csv')

In [27]:
## Export into a csv file format
column_names = ['ticker', 'quarter_end', 'notice', 'revenue', 'gross_profit', 'operating_profit', 'net_income']
quarterEnd = pd.DataFrame(data=quarterEnd, columns=column_names)
quarterEnd.to_csv('./datasets/company_quarter_ends.csv')

In [28]:
## Export into a csv file format
column_names = ['ticker', 'ratings']
ratings = pd.DataFrame(data=ratings, columns=column_names)
ratings.to_csv('./datasets/company_ratings.csv')